# Parse HLS families notices

The overall goal is to "clean" notices by removing html, links, abreviations etc from the notices. 

In [15]:
import re
import geovpylib.database as db
from gmpykit.ipython import infos
import gmpykit as kit


eta = kit.Eta()
db.connect_yellow('switzerland_and_beyond', execute=True)

[DB] Connecting to YELLOW database "switzerland_and_beyond" ... Connected!


## Fetch the data

In [16]:
families = db.query('select * from hls.family')
infos(families)

Shape:  (2539, 5) - extract:


,id,url,name,notice_html,notice
0,998,https://hls-dhs-dss.ch/fr/articles/022780,Enk,"Vieille famille d'Altstätten, dont la première...",None
1,18,https://hls-dhs-dss.ch/fr/articles/019509,"Bechburg, von","Famille noble, attestée entre 1100 environ et ...",None
2,19,https://hls-dhs-dss.ch/fr/articles/019980,"Beuggen, von",Famille de petite noblesse mentionnée dans la ...,None
3,29,https://hls-dhs-dss.ch/fr/articles/003013,"Amman, d'","Famille patricienne originaire de Neyruz (FR),...",None
4,38,https://hls-dhs-dss.ch/fr/articles/022042,Belorsier,"Famille bourgeoise de Saint-Ursanne, dont sont...",None


## Update the table

In [17]:
if "notice" not in families.columns:
    db.connect_yellow('switzerland_and_beyond', execute=True)
    db.execute('alter table hls.person add notice varchar;')
    db.connect_yellow('switzerland_and_beyond', execute=False)

## Clean the HTML

In [18]:
def handle_exponent(notice):
    begin_words = '<sup><span locale="fr">'
    end_words = '</span></sup>'

    while '<sup>' in notice:
        try:
            begin = notice.index(begin_words)
            end = notice.index(end_words, begin) + len(end_words)
            origin = notice[begin:end]
            result = origin.replace(begin_words, '').replace(end_words, '')
            notice = notice.replace(origin, result)
        except: notice = notice
    return notice

def handle_ampersand(notice):
    return notice.replace('&amp;', '&')


def handle_italics(notice):
    begin_words = '<em><span locale="fr">'
    end_words = '</span></em>'

    while '<em>' in notice:
        try:
            begin = notice.index(begin_words)
            end = notice.index(end_words, begin) + len(end_words)
            origin = notice[begin:end]
            result = origin.replace(begin_words, '').replace(end_words, '')
            notice = notice.replace(origin, result)
        except: notice = notice
    return notice

def replace_chars(notice):
    notice = notice.replace('&nbsp;', ' ')
    notice = notice.replace('&ZeroWidthSpace;', ' ')
    notice = notice.replace('\u200b', '')
    return notice

def expand_shortcuts(notice):
    notice = notice.replace('cath.-chr.', 'catholique chrétien')
    notice = notice.replace('cath.', 'catholique')
    notice = notice.replace('prot.', 'protestant')
    notice = notice.replace('comm.', 'commune de')
    notice = notice.replace('distr.', 'district')
    notice = notice.replace('Comm.', 'commune de')
    notice = notice.replace('auj.', 'aujourd\'hui')
    notice = notice.replace(' s.', ' siècle')
    notice = notice.replace(' hab.', ' habitants')
    notice = notice.replace(' franç.', ' français')
    return notice


def handle_links(notice):
    begin_words = '<span class="wikilink">'
    end_words = '</span>'

    while 'href' in notice:
        try:
            begin = notice.index(begin_words)
            end = notice.index(end_words, begin) + len(end_words)
            href_begin = notice.index('"', notice.index('href')) + 1
            href_end = notice.index('"', href_begin)
            origin = notice[begin:end]

            link_content_begin = origin.index('>', origin.index('<a ')) + 1
            link_content_end = origin.index('</a>')
            link_content = origin[link_content_begin:link_content_end].strip()
            link_content = link_content.replace('-&gt;', '')
            if not link_content == '': 
                link_content += ' ('
                suffix = ')'
            else: suffix = ''
            notice = notice.replace(origin, link_content + "https://hls-dhs-dss.ch" + notice[href_begin:href_end] + suffix)
        except: 
            pass
    
    return notice


def expand_name(notice, name):
    unique_caps_with_dot = re.findall(r'\s([A-Z])\.', notice)
    for caps in unique_caps_with_dot:
        notice = notice.replace(f' {caps}. ', ' ' + name + ' ')
        notice = notice.replace(f' {caps}.,', ' ' + name + ' ')
    return notice.replace(f'-{name[0:1]}.', f'-{name}')

def handle_notice(name, notice):

    notice = notice.replace('wikiinternallink', 'wikilink')
    
    notice = handle_italics(notice)
    notice = replace_chars(notice)
    notice = handle_exponent(notice)
    notice = handle_ampersand(notice)
    notice = expand_shortcuts(notice)
    notice = handle_links(notice)
    notice = expand_name(notice, name)
    

    return notice.strip()

In [19]:
eta.begin(len(families), "Cleaning notices")
for i, row in families.iterrows():
    cleaned = handle_notice(row['name'], row['notice_html'])
    db.execute(f"""
        update hls.family
            set notice = '{cleaned.replace("'", "''")}'
        where id = {row['id']};
    """)
    eta.iter()
eta.end()

Cleaning notices - 2539 iterations in 00h01m14s (avg of 00h00m00s/iter)               


___

In [ ]:
notice = families.sample(1).iloc[0]

print(kit.wrap(handle_notice(notice['name'], notice['notice_html'])))

In [ ]:
print(kit.wrap(handle_notice(notice['name'], notice['notice_html'])))

In [ ]:
notice